In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
import pandas as pd

In [ ]:
# papermill parameters
output_folder = "../output/"

## Functions

In [ ]:
def find_potential_divs(soup):
    divs = soup.findAll(lambda tag:tag.name == "div" and len(tag.attrs) == 2)
    found_div = []
    for div in divs:
        if 'data-app-js' in div.attrs.keys():
            found_div.append(div)
    return found_div

def parse_state_actions(div):
    return eval(div['data-app-js'].replace(
    'false', 'False').replace('true', 'True').replace(
    'null', 'None'))['gdocs_object']

def get_df_from_parsed(parsed):
    df = pd.DataFrame(parsed[0][1])
    df.loc[0, 0] = 'Location'
    df.columns = df.loc[0]
    df = df.drop([0,1, 2])
    df = df.reset_index(drop=True)
    return df

## Get request and parse response

In [ ]:
r = requests.get("https://www.kff.org/health-costs/issue-brief/state-data-and-policy-actions-to-address-coronavirus/")

In [ ]:
soup = BeautifulSoup(r.content, 'html5lib')

## Get tables

In [ ]:
found_divs = find_potential_divs(soup)

### State Actions to Mitigate the Spread of COVID-19

In [ ]:
parsed = parse_state_actions(found_divs[3])

In [ ]:
df_actions = get_df_from_parsed(parsed)
df_actions["Last_Update_Date"] = datetime.utcnow()

In [ ]:
df_actions.to_csv(f'{output_folder}/KFF_US_STATE_MITIGATIONS.csv', index=False)

### State COVID-19 Health Policy Actions

In [ ]:
parsed = parse_state_actions(found_divs[4])

In [ ]:
df_health = get_df_from_parsed(parsed)
df_health["Last_Update_Date"] = datetime.utcnow()

In [ ]:
df_health.to_csv(f'{output_folder}/KFF_US_POLICY_ACTIONS.csv', index=False)